# Installazione librerie e configurazione


In [ ]:
!git clone https://github.com/agwmon/MuDI.git
%cd MuDI

In [ ]:
!pip install diffusers[torch] transformers peft wandb scipy

!pip install git+https://github.com/facebookresearch/segment-anything.git

In [ ]:
#@title Inserire il token per loggarsi su Hugginface 🤗

#@markdown Serve per scaricare correttamente i pesi di Stable Diffusion ed eventualmente salvarsi il modello dopo
!mkdir -p ~/.huggingface
HUGGINGFACE_TOKEN = ""  #@param {type:"string"}
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

In [ ]:
#@title Upload delle: reference_photo, maschere e il jsonl

#@markdown Inserisci il percorso in cui salvare i file (verrà creato se non esiste):
save_path = "./dataset/category/andrea_cami"  #@param {type:"string"}

import os
from google.colab import files

# Crea la directory se non esiste
os.makedirs(save_path, exist_ok=True)

# Prompt di upload
uploaded = files.upload()  # Salva automaticamente i file in /content

# Copia in save_path e rimuovi i file temporanei da /content
for filename, filecontent in uploaded.items():
    src_path = os.path.join("/content", filename)
    dest_path = os.path.join(save_path, filename)

    # Scrivi nella destinazione
    with open(dest_path, 'wb') as f:
        f.write(filecontent)
    print(f"✅ Salvato: {dest_path}")

    # Elimina il file temporaneo originale da /content
    if os.path.exists(src_path):
        os.remove(src_path)



In [ ]:
#@title Se si vuole utilizzare wandb si specifichi la chiave di utilizzo

#@markdown ###  Vuoi usare Weights & Biases (wandb)?
use_wandb = True  #@param {type:"boolean"}

import os

if use_wandb:
    #@markdown Inserisci il tuo API Key wandb (obbligatoria se `use_wandb` è True)
    wandb_api_key = ""  #@param {type:"string"}

    os.environ["WANDB_API_KEY"] = wandb_api_key
    import wandb
    wandb.login(key=wandb_api_key)
    print("✅ wandb attivato e autenticato.")
else:
    os.environ["WANDB_MODE"] = "disabled"
    print("🚫 wandb disattivato.")



#Train

In [ ]:
VAL_PROMPT="a photo of anrosc person and camdndr person walking together"
DATASET_NAME="andrea_cami"
SEG_MIX_PROB=0.3
SEG_MIX_START_STEP=0

!accelerate launch train_segmix_lora_sdxl.py --instance_data_dir="dataset/category/"$DATASET_NAME \
    --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
    --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" \
    --output_dir=output/$DATASET_NAME"_p"$SEG_MIX_PROB"_s"$SEG_MIX_START_STEP \
    --validation_prompt="$VAL_PROMPT" --report_to=wandb --resolution=1024 --train_batch_size=1 --gradient_accumulation_steps=2 \
    --checkpointing_steps=2000 --max_train_steps=4100 --validation_epochs=1500 --save_steps=1000 --lr_warmup_steps=0 --seed=42 --rank=32 --learning_rate=1e-5 \
    --segmix_prob=0.3 --segmix_start_step=0 --relative_scale=0.0 --soft_alpha=1.0 \
    --gradient_checkpoint


# SEG_MIX_PROB rappresenta la probabilità con cui viene applicato segmix ad ogni step
# SEG_MIX_START_STEP è da quale step iniziare segmix
# I numero di passi e il lr pensiamo debba essere rispettivamente sui 4000 e nell'ordine di e-5
# relative_scale e soft_alpha sono altri parametri che possono servire durante l'esecuzione del segmix. Il primo serve per riproproporzionare le reference fornite se 0 le due reference
# rimangono uguali se 0.5 la reference a sarà al 100% e la b al 50%. Il secondo serve per sfumare le due maschere durante la loro fusione.

In [ ]:
#@title Salvataggio della cartella prodotta con tutti i LORA generati

folder_path = "output/" + DATASET_NAME

import os
from google.colab import files

# Verifica se la cartella esiste
if not os.path.isdir(folder_path):
    raise FileNotFoundError(f"La cartella '{folder_path}' non esiste.")

# Comprimi la cartella in un file zip
zip_filename = os.path.basename(folder_path.rstrip("/")) + ".zip"
zip_path = f"/content/{zip_filename}"
!zip -r -q "{zip_path}" "{folder_path}"

# Scarica il file zip localmente
files.download(zip_path)
